In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
# We generate a document id for each document. We we also use a combination of course + question,
# but that create a few duplicate hashes where the course and question were exactly same on the FAQ.
# Therefore we used the text field as well and extracted the first 10 characters from it.
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [3]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [5]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [7]:
from collections import defaultdict

In [8]:
# This creates a default dictionary which is a list
hashes = defaultdict(list)
print("hashes : ", hashes)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

hashes :  defaultdict(<class 'list'>, {})


In [9]:
len(hashes), len(documents)

(947, 948)

In [12]:
hashes.items()

dict_items([('c02e79ef', [{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp', 'id': 'c02e79ef'}]), ('1f6520ca', [{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites', 'section': 'General course-related questions', 'question': 'Course - What are the prerequisites for this course?', 'course': 'data-engineering-zoomcamp', 'id': '1f6520ca'}]), ('7842b56a', [{'text': "Yes, even if you don't register, you're still elig

In [13]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [14]:
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [15]:
import json

In [16]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [17]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [58]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks. Do not add any text other than the list of questions:

["question1", "question2", ..., "question5"]
""".strip()

In [28]:
documents

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'id': 'c02e79ef'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'id': '1f6520ca'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware

In [67]:
# 
doc = documents[2]
prompt = prompt_template.format(**doc)

In [68]:
print(prompt)

You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Provide the output in parsable JSON without using code blocks. Do not add any text other than the list of questions:

["question1", "question2", ..., "question5"]


In [69]:
# Since I don't have OpenAI I will use Groq. But then to sync up with the module lecture, I will use the results from the 
# binary file "results.bin"
# from openai import OpenAI
# client = OpenAI()

# The generated questions are different between the 2 models though they are more or less trying to be similar to the 
# question from FAQ
from groq import Groq
import os
import json
# api_key = os.environ.get("GROQ_API_KEY")
client = Groq(api_key=api_key)

response = client.chat.completions.create(
    model='llama3-8b-8192', # We are using llama3 within Groq instead of 'gpt-4o'
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message

json_response = response.choices[0].message.content
json_response

json.loads(json_response)

['Can I still take the course after the start date? What are the deadlines for final projects?',
 'How do late submissions of homeworks affect the final grade?',
 'Are there any penalties for missing the deadlines for final projects?',
 'What consequences can I expect if I leave everything for the last minute?',
 'Can I still join the course halfway through and still finish all the homeworks?']

In [70]:
print(json_response)

["Can I still take the course after the start date? What are the deadlines for final projects?", "How do late submissions of homeworks affect the final grade?", "Are there any penalties for missing the deadlines for final projects?", "What consequences can I expect if I leave everything for the last minute?", "Can I still join the course halfway through and still finish all the homeworks?"]


In [75]:
# We will run this function using groq
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    # response = client.chat.completions.create(
    #     model='gpt-4o',
    #     messages=[{"role": "user", "content": prompt}]
    # )
    response = client.chat.completions.create(
    model='llama3-8b-8192', # We are using llama3 within Groq instead of 'gpt-4o'
    messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [76]:
from tqdm.auto import tqdm

In [77]:
results = {}

In [78]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results: # Sometimes the for loop will fail if the rate limit is hit. therefore we skip those 
                          # doc_id next time this step is re-run
        continue
    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/948 [00:00<?, ?it/s]

In [79]:
results

{'c02e79ef': '["When will the course start, exactly?", "What is the purpose of this document?", "When does the \'Office Hours\' live start?\', "How do I subscribe to the course public Google Calendar?", "How do I register before the course starts?"]',
 '1f6520ca': '["What are the prerequisites for this course that I need to take before joining the data engineering zoomcamp course?", "What if I don\'t have experience in data engineering yet?", "Can I take this course as a beginner?", "Are there any specific skills I need for this course?", "Do I need to have GitHub for this course?"]',
 '7842b56a': '["Can I still register for the course if I missed the registration deadline?",\n"Are there any deadlines for finishing assignments?",\n"What if I miss the first few classes, will that affect my submissions?",\n"Can I still join the course if I didn\'t attend the first class?",\n"Should I plan my final project completion in advance?"]',
 '0bbf41ec': '["When will I receive the confirmation ema

In [80]:
results['1f6520ca']

'["What are the prerequisites for this course that I need to take before joining the data engineering zoomcamp course?", "What if I don\'t have experience in data engineering yet?", "Can I take this course as a beginner?", "Are there any specific skills I need for this course?", "Do I need to have GitHub for this course?"]'

In [81]:
import pickle

In [83]:
# I will now write dictionary into a pickle file for retrieval later
with open('results_llama3_groq.bin', 'wb') as file:
    pickle.dump(results, file)

In [87]:
# I will now use the binary file generated by Alexey from OpenAI
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [88]:
results['1f6520ca']

'["Where can I find the prerequisites for this course?", "How do I check the prerequisites for this course?", "Where are the course prerequisites listed?", "What are the requirements for joining this course?", "Where is the list of prerequisites for the course?"]'

In [89]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)

JSONDecodeError: Invalid \escape: line 6 column 59 (char 414)

In [90]:
print(json_questions)

[
"How can I resolve the Docker error 'invalid mode: \\Program Files\\Git\\var\\lib\\postgresql\\data'?",
"What should I do if I encounter an invalid mode error in Docker on Windows?",
"What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",
"Can you provide an example of a correct Docker mounting path for PostgreSQL data?",
"How do I correct the mounting path error in Docker for \\\Program Files\\Git\\var\\lib\\postgresql\\data'?"
]


In [91]:
# The above record is fixed manually by chahing the 2 forward slash to 1
json_questions = [
r"How can I resolve the Docker error 'invalid mode: \Program Files\Git\var\lib\postgresql\data'?",
"What should I do if I encounter an invalid mode error in Docker on Windows?",
"What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",
"Can you provide an example of a correct Docker mounting path for PostgreSQL data?",
r"How do I correct the mounting path error in Docker for \Program Files\Git\var\lib\postgresql\data'?"
]

In [92]:
print(doc_id)

58c9f99f


In [93]:
json.dumps(json_questions)

'["How can I resolve the Docker error \'invalid mode: \\\\Program Files\\\\Git\\\\var\\\\lib\\\\postgresql\\\\data\'?", "What should I do if I encounter an invalid mode error in Docker on Windows?", "What is the correct mounting path to use in Docker for PostgreSQL data on Windows?", "Can you provide an example of a correct Docker mounting path for PostgreSQL data?", "How do I correct the mounting path error in Docker for \\\\Program Files\\\\Git\\\\var\\\\lib\\\\postgresql\\\\data\'?"]'

In [94]:
results[doc_id]=json.dumps(json_questions)

In [95]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)

In [96]:
doc_index = {d['id']: d for d in documents}

In [97]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [98]:
import pandas as pd

In [99]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [100]:
df.to_csv('ground-truth-data.csv', index=False)

In [101]:
!head ground-truth-data.csv

question,course,document
When does the course begin?,data-engineering-zoomcamp,c02e79ef
How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
Where can I find the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
How do I check the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
Where are the course prerequisites listed?,data-engineering-zoomcamp,1f6520ca
What are the requirements for joining this course?,data-engineering-zoomcamp,1f6520ca
